In [1]:
%run /Users/tyamgin/Projects/mlbootcamp/championship19/cv.ipynb
input_path = '/Users/tyamgin/Projects/mlbootcamp/championship17/data'
output_path = '/Users/tyamgin/Projects/mlbootcamp/championship17/res'

/usr/local/lib/python3.7/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)
Using TensorFlow backend.


In [2]:
train_data = parquet.read_table(input_path + '/collabTrain').to_pandas()

/usr/local/lib/python3.7/site-packages/pyarrow/pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


In [3]:
test_data = parquet.read_table(input_path + '/collabTest').to_pandas()

In [39]:
for c in ['HAS_CHEATED_TEXT', 'HAS_COMPANIONS', 'HAS_DETECTED_TEXT',
       'HAS_MUSIC', 'HAS_PHOTOS', 'HAS_PINS', 'HAS_POLLS', 'HAS_TEXT',
       'HAS_URLS', 'HAS_VIDEOS', 'IS_CENSORED', 'IS_DISGUSTING',
       'IS_EXTERNAL_SHARE', 'IS_GIF', 'IS_INTERNAL_GROUP_SHARE',
       'IS_INTERNAL_SHARE', 'IS_PART_OF_ALBUM', 'IS_PART_OF_TOPIC',
       'IS_PROMO']:
    c = c.lower()
    print(c, train_data[c].sum())

KeyError: 'has_cheated_text'

In [38]:
train_data.owner_gender.isna().mean()

1.0

In [31]:
len(train_data.metadata_authorId.unique()), len(test_data.metadata_authorId.unique()),\
    len(np.intersect1d(train_data.metadata_authorId.unique(), test_data.metadata_authorId.unique())) 

(298567, 64490, 48484)

In [32]:
len(train_data.metadata_ownerId.unique()), len(test_data.metadata_ownerId.unique()),\
    len(np.intersect1d(train_data.metadata_ownerId.unique(), test_data.metadata_ownerId.unique())) 

(81272, 41917, 41321)

In [35]:
train_data.columns

Index(['instanceId_userId', 'instanceId_objectType', 'instanceId_objectId',
       'audit_pos', 'audit_clientType', 'audit_timestamp', 'audit_timePassed',
       'audit_experiment', 'audit_resourceType', 'metadata_ownerId',
       ...
       'auditweights_userOwner_USER_PRESENT_SEND',
       'auditweights_userOwner_USER_PROFILE_VIEW',
       'auditweights_userOwner_USER_SEND_MESSAGE',
       'auditweights_userOwner_USER_STATUS_COMMENT_CREATE',
       'auditweights_userOwner_VIDEO', 'auditweights_userOwner_VOTE_POLL',
       'auditweights_x_ActorsRelations', 'auditweights_likersSvd_spark_hyper',
       'auditweights_source_PROMO', 'date'],
      dtype='object', length=169)

In [36]:
train_data[['instanceId_objectId', 'instanceId_userId', 'date']]\
    .groupby(['instanceId_objectId', 'instanceId_userId']).count()

date
instanceId_objectId instanceId_userId      
4                   14662188              1
5                   4119048               1
                    7471560               1
                    13158108              1
6                   3903447               1
                    6672564               1
                    7873182               1
                    11547906              1
9                   833643                1
                    1678692               1
                    1999902               1
                    7170642               1
                    7891104               1
12                  1228218               1
                    4726368               1
14                  8272740               1
                    11926803              1
15                  7602081               1
                    14804529              1
                    15318849              1
16                  5537859               1
17                  3931866               1
                    7488945               1
                    7988172               1
                    9145602               1
                    12289425              1
                    13216641              1
                    13450647              1
                    14722692              1
18                  6250248               1
                    15229650              1
20                  4260540               1
                    5764884               1
22                  10696362              1
23                  5323878               1
                    7592832               1
                    8687313               1
                    13515030              1
                    14644206              1
24                  323940                1
                    4889385               1
26                  3933921               1
                    15391452              1
27                  13289559              1
30                  9517095               1
31                  6681636               1
33                  7320402               1
                    8767686               1
                    10191063              1
                    10900887              1
                    11748540              1
                    12668682              1
                    12913704              1
                    13509492              1
                    14987220              1
                    15483969              1
35                  12154086              1
36                  3621495               1
37                  1260081               1
                    3665406               1
...                                     ...
39082211            6052047               1
                    7463658               1
                    7657986               1
                    9869034               1
                    10283553              1
                    10757982              1
                    10915434              1
                    10928316              1
                    13866591              1
39082212            1445046               1
                    4816284               1
                    5100579               1
                    5365242               1
                    8707692               1
                    10092690              1
39082213            4617387               1
39082214            839091                1
                    2246871               1
                    5290146               1
                    6752817               1
                    11835654              1
                    14553885              1
                    15484602              1
39082215            6721947               1
                    8573862               1
                    8587686               1
                    8712744               1
                    8883375               1
                    12562590       

In [8]:
def proc_data_inplace(data):  
    data['instanceId_objectType'] = np.array(data.instanceId_objectType.replace({
        'Post': 0,
        'Video': 1,
        'Photo': 2,
    }), np.int8)
    data['audit_clientType'] = np.array(data.audit_clientType.replace({
        'WEB': 0,
        'API': 1,
        'MOB': 2,
    }), np.int8)
    data['metadata_ownerType'] = np.array(data.metadata_ownerType.replace({
        'GROUP_OPEN_OFFICIAL': 0,
        'GROUP_OPEN': 1,
    }), np.int8)
    data['metadata_platform'] = np.array(data.metadata_platform.replace({
        'OTHER': 0,
        'WEB': 1,
        'ANDROID': 2,
        'IOS': 3,
        'MOB': 4,
        'WINPHONE': 5,
    }), np.int8)

    drop_columns = [
        # nan or single value, == userOwnerCounters_USER_PROFILE_VIEW
       'userOwnerCounters_VOTE_POLL',
       'userOwnerCounters_USER_SEND_MESSAGE',
       'userOwnerCounters_USER_DELETE_MESSAGE',
       'userOwnerCounters_USER_INTERNAL_LIKE',
       'userOwnerCounters_USER_INTERNAL_UNLIKE',
       'userOwnerCounters_USER_STATUS_COMMENT_CREATE',
       'userOwnerCounters_PHOTO_COMMENT_CREATE',
       'userOwnerCounters_MOVIE_COMMENT_CREATE',
       'userOwnerCounters_USER_PHOTO_ALBUM_COMMENT_CREATE',
       'userOwnerCounters_COMMENT_INTERNAL_LIKE',
       'userOwnerCounters_USER_FORUM_MESSAGE_CREATE',
       'userOwnerCounters_PHOTO_MARK_CREATE',
       'userOwnerCounters_PHOTO_VIEW',
       'userOwnerCounters_PHOTO_PIN_BATCH_CREATE',
       'userOwnerCounters_PHOTO_PIN_UPDATE',
       'userOwnerCounters_USER_PRESENT_SEND',

       # single value
       'metadata_applicationId',
        
       # bad col in test
       'auditweights_closed',
    ]
    
    for c in data.columns:
        print(c)
        if data[c].isna().all():
            drop_columns.append(c)
    print('Drop columns: %s' % str(drop_columns))
    data.drop(drop_columns, 1, inplace=True)
        
    for c in ('HAS_CHEATED_TEXT', 'HAS_COMPANIONS', 'HAS_DETECTED_TEXT',
       'HAS_MUSIC', 'HAS_PHOTOS', 'HAS_PINS', 'HAS_POLLS', 'HAS_TEXT',
       'HAS_URLS', 'HAS_VIDEOS', 'IS_CENSORED', 'IS_DISGUSTING',
       'IS_EXTERNAL_SHARE', 'IS_GIF', 'IS_INTERNAL_GROUP_SHARE',
       'IS_INTERNAL_SHARE', 'IS_PART_OF_ALBUM', 'IS_PART_OF_TOPIC',
       'IS_PROMO'):
        data[c.lower()] = np.array(data.metadata_options.apply(lambda x: 1 if c in x else 0).values, np.int8)
    data.drop('metadata_options', 1, inplace=True)
    
    if 'feedback' in data.columns:
        for c in ('Liked', 'Clicked', 'Viewed', 'Disliked', 'ReShared', 'Ignored', 'Commented', 'Complaint', 'Unliked'):
            data[c.lower()] = np.array(data['feedback'].apply(lambda x: 1 if c in x else 0).values, np.int8)
        data.drop('feedback', 1, inplace=True)
      
    data['membership_status'] = np.array(data.membership_status.replace({
        'A': 1,
        'P': 2,
        'I': 3,
        'Y': 4,
        'M': 5,
        'B': 6,
        'R': 7,
        '!': 8,
    }).fillna(0), np.int8)
    
    for c, ctype in zip(data.columns, data.dtypes):
        if ctype == 'float64':
            data[c] = np.array(data[c], np.float32)
    

    

In [9]:
proc_data_inplace(train_data)

In [10]:
proc_data_inplace(test_data)

In [2]:
train_data = pd.read_pickle(output_path + '/train_data')
test_data = pd.read_pickle(output_path + '/test_data')

In [11]:
test_data.dtypes

instanceId_userId                                 int32
instanceId_objectType                              int8
instanceId_objectId                               int32
audit_pos                                         int64
audit_clientType                                   int8
audit_timestamp                                   int64
audit_timePassed                                  int64
audit_experiment                                 object
audit_resourceType                                int64
metadata_ownerId                                  int32
metadata_ownerType                                 int8
metadata_createdAt                                int64
metadata_authorId                                 int32
metadata_applicationId                            int64
metadata_numCompanions                            int32
metadata_numPhotos                                int32
metadata_numPolls                                 int32
metadata_numSymbols                             

In [25]:
train_data.metadata_ownerType.unique()

array(['GROUP_OPEN_OFFICIAL', 'GROUP_OPEN'], dtype=object)

In [27]:
train_data.metadata_platform.unique()

array(['OTHER', 'WEB', 'ANDROID', 'IOS', 'MOB', 'WINPHONE'], dtype=object)

In [69]:
train_data.membership_status.unique()

array(['A', None, 'P', 'I', '!', 'Y', 'M', 'B', 'R'], dtype=object)

In [45]:
np.unique(np.concatenate(train_data.metadata_options.values))

array(['HAS_CHEATED_TEXT', 'HAS_COMPANIONS', 'HAS_DETECTED_TEXT',
       'HAS_MUSIC', 'HAS_PHOTOS', 'HAS_PINS', 'HAS_POLLS', 'HAS_TEXT',
       'HAS_URLS', 'HAS_VIDEOS', 'IS_CENSORED', 'IS_DISGUSTING',
       'IS_EXTERNAL_SHARE', 'IS_GIF', 'IS_INTERNAL_GROUP_SHARE',
       'IS_INTERNAL_SHARE', 'IS_PART_OF_ALBUM', 'IS_PART_OF_TOPIC',
       'IS_PROMO'], dtype=object)

In [70]:
np.unique(np.concatenate(train_data.feedback.values))

array(['Clicked', 'Commented', 'Complaint', 'Disliked', 'Ignored',
       'Liked', 'ReShared', 'Unliked', 'Viewed'], dtype=object)

In [ ]:
for c in train.columns:
    test_uniq = test[c].unique() if c in test.columns else np.array([1,2])
    train_uniq = train[c].unique()

    if len(train_uniq) == 1 or len(test_uniq) == 1:
        print('all ' + c)
    elif (~np.isnan(train_uniq)).sum() == 1 or (~np.isnan(test_uniq)).sum() == 1:
        print('all2 ' + c)

In [5]:
train_data.to_pickle(output_path + '/train_data')
test_data.to_pickle(output_path + '/test_data')